<img src="images/one_logo.png" width="500"/> 

# Polytope requests examples - Perturbed Ensemble Surface forecast data

This notebook holds example request for getting Surface forecast data from ECMWF's Ensemble forecast. 

The difference between deterministic and ensemble forecast is that Ensemble has 50 perturbed members. The data is organised in a different 'stream' which is **enfo** in this case, and we hav a new parameter 'number', which represent each ensemble member number (1-50).

Before we can run the requests we need to import polytope (and xarray to later inspect the data) and start the client.

In [ ]:
!python3 -m pip install --upgrade git+https://git.ecmwf.int/scm/lex/polytope-client.git@master

If you're working on **mybinder** you should already have **xarray, matplotlib** and **cfgrib** installed.  
If you're working locally and don't have them installed, please run the next cell.

In [ ]:
import sys
!conda install --yes --prefix {sys.prefix} xarray cfgrib matplotlib

In [ ]:
from polytope.api import Client

In [ ]:
import xarray as xr

Insert here your email and ecmwf token, or username and password:

In [ ]:
c = Client(user_email = 'johndoe@ecmwf.int',
           user_key = '4j3s3d34n4sn335jacf3n3d4f4g61635')

In [ ]:
c = Client(username = 'test1', password = 'password1')

In [ ]:
c.list_collections()

### Get 2m temperature - individual members
This example request will download 2m temperature data for all 50 ensemble members. 

ECMWF ENSemble forecast data can be found in stream **enfo** (**en**semble **fo**recast).  
Individual perturbed members have type **pf** (**p**erturbed **fo**ecast).  
New keword here is **number** which can have values 0-50. Each number represent one ensemble member.  
Syntax is similar to step or levelist syntax.

Full list of available SFC ensemble parameters you can find in archive catalogue on this link:  
https://apps.ecmwf.int/archive-catalogue/?stream=enfo&levtype=sfc&time=00%3A00%3A00&expver=1&month=mar&year=2020&date=2020-03-08&type=pf&class=od  
You can also see available steps there.  
Data is available on 0.2x0.2 resolution, but we put 1x1 here to get smaller file for the example. 

In [ ]:
request1 = {
    'stream': 'enfo',
    'number': '1/to/10',
    'levtype': 'sfc',
    'param': '2t',
    'step': '0/3/6/9/12',
    'time': '00',
    'date': '20200305',
    'type': 'pf',
    'class': 'od',
    'expver': '0001',
    'area': '50/15/40/25',
    'grid': '1/1'
}

To retrieve the data just run the retrieve.

In [ ]:
c.retrieve('hidalgo-test', request1, 'data/pf_sfc.grib')

In [ ]:
ds = xr.open_dataset('data/pf_sfc.grib',engine='cfgrib')
ds

There is a 51st member of our ensemble. It is so called Control forecast. This is the unperturbed member with the same configuration of model as high resolution forecast.  
This data has type **cf** and since different types can't be mixed in the same request we have to write separate request for this data.  

Everything is the same except **type = cf** and we need to remove number keyword.

In [ ]:
request2 = {
    'stream': 'enfo',
    'levtype': 'sfc',
    'param': '2t',
    'step': '0/3/6/9/12',
    'time': '00',
    'date': '20200305',
    'type': 'cf',
    'class': 'od',
    'expver': '0001',
    'area': '50/15/40/25',
    'grid': '1/1'
}

Now we can run these 2 reqeusts together and save the output in the same file.

In [ ]:
c.retrieve('hidalgo-test', [request1,request2], 'data/ens_sfc.grib')

If we inspect data now, we can notice that xarray and cfgrib can't handle grib file that has 2 diferent **types**.  
But we can still read this grib using filter_by_keys and filter each type. Uncomment second or third row to load pf or cf data.

In [ ]:
ds = xr.open_dataset('data/sfc.grib',engine='cfgrib')
#ds = xr.open_dataset('sfc.grib',engine='cfgrib', backend_kwargs={'filter_by_keys': {'dataType': 'pf'}})
#ds = xr.open_dataset('sfc.grib',engine='cfgrib', backend_kwargs={'filter_by_keys': {'dataType': 'cf'}})
ds